<h1 style="text-align:center">Kaggle Competition Report</h1>

<h3 style="text-align:center">
Hayden Bader,
Yujing Ke,
Yangdi Shen</h3>

## Abstract

High-resolution satellite imaging combined with image-detection and machine-learning techniques have previously been shown to be very effective at determining whether identifiable structures, such as solar panels, exist within a localized region. This report was conducted in conjunction with analyses promoted by a Kaggle prediction competition, Island in the Sun. Ultimately, the technique used for the final submission was a K-Nearest Neighbors model using features primarily analyzed in the HSV color space to achieve an ROC curve with an estimated 0.82 AUC. Though this result does not improve upon prior work, it analyzes a data set comprising nearly 1500 images and reports additional information about the efficacy of various alternate techniques. 


## Introduction

Over the past several years, public and economic concerns have increased interest and investment in “greener” technologies that seek to augment renewable energy production while decreasing dependence on finite energy resources or potentially deleterious forms of energy production. To this end, improving the efficiency of solar panels has been the subject of much research from energy companies who seek to produce the most desirable solar arrays for both public and private consumption. In a similar manner, there has been much public policy research that has been conducted into the efficacy of solar panel placement, clustering, and maintenance among other concerns (Castellanos, 2017).

However, in order to make effective policy decisions about the future use of localized solar panels, each municipality has to consider how its individual regional features will impact solar panel effectiveness. Furthermore, as solar array installation has  attempting to form a policy must have some way of analyzing data obtainable from both individual and industrial solar arrays. Furthermore, in order to retrieve information about individuals using solar arrays, analysts would typically have to either have to hope to be able to retrieve information reported directly by users or information that can be extracted from local energy company reports. Gathering this information can be time-consuming, and worse, additional developments can quickly cause regional information to become obsolete.

As a result, researchers have sought less time and resource intensive ways to gather relevant information about solar arrays. This paper summarizes some of the previous work that has gone into this problem as well as presents and evaluates an attempt at a generalizable model that may be useful in similar future endeavors. 



## Background

Using high-definition satellite images to identify solar panels has been shown to be effective on a sample size of 100 satellite images obtained from the US Geological Survey (Malof, 2015). In order to attain their results, the aforementioned study sought to identify Maximally Stable Extremal Regions (MSER) to prescreen parts of their dataset for further analysis. Afterwards, the images that were deemed to exhibit a region of interest were analyzed further, and additional features including background color composition, primary region color, and region shape were extracted. A support vector machine learning classifier with manually labeled training data was then used to generate estimates on testing data to determine the effectiveness of this approach. The result was a 94% detection rate of true panels within the initially obtained images. 

The methods presented in this paper are largely built upon those discussed in Automatic Solar Photovoltaic Panel Detection in Satellite Imagery (Malof, 2015) with a few key differences that will be discussed in more depth below. 

Contextually, though, this report is the result of a Kaggle Competition that seeks to have students apply their understanding of machine learning concepts to attempt to build an effective classifier.  This particular competition was titled Island in the Sun, contained a set of 1500 high-resolution satellite images for use as training data. Additional separate images were used for both localized testing and global evaluation. Images containing solar panels came prelabeled, so labeling strategies or potential errors are not considered in this report. 


## Data

The original training data consists of 1500 separated satellite images from the map. Each image contains 101x101 pixels information. Among all training images, we have 995 images which does not include PV solar panels while 505 images contain PV solar panels. We could use *Imread* function in the matplotlib package to transform every image into 101x101x3 vector which consists of R(red), G(green), B(blue) value of every pixel in the image. Therefore, we could get a numpy array with a dimension of 1500x101x101x3. This information is original data that we could extract from the images. In addition, we could transform image’s RGB data into HSV (Hue, Saturation, Value) format through *rgb_to_hsv* function.  As the alternative representation of RGB color model, HSV could provide us with some different points of view. (We would transform them into the same scale latter) The following figure gives some sample images in both RGB model(original) and HSV model (transformed).

![image](https://raw.githubusercontent.com/ys203/machine-learning-course/master/image.png)
<center>Figure 1. Sample image in RGB and HSV scales<center>


As we could see in the sample images above, RGB and HSV present the same image in two diverse ways. In recognition process of most images, the existence of tree, roof, road, even shadow could interfere our classification process. Therefore, we need to use some features to distinguish PV solar panels with other things. We could look at the general data distribution of R,G,B,H,S,V values of all images by plotting their mean value and standard deviation.


![image](https://raw.githubusercontent.com/ys203/machine-learning-course/master/Mean_RGB.png)
![image](https://raw.githubusercontent.com/ys203/machine-learning-course/master/Std_RGB.png)
![image](https://raw.githubusercontent.com/ys203/machine-learning-course/master/Mean_hsv.png)
![image](https://raw.githubusercontent.com/ys203/machine-learning-course/master/Std_hsv.png)

<center>Figure 2, 3, 4, 5 . RGB & HSV mean value and sd distribution for images<center>


As shown in the graphs above, the general distributions of those data for images including PV solar panels and those not are not significantly different. (except the standard deviation of h shows a slight excess for those include) It means that we should extract more features which could better distinguish PV solar panels from inference stuff or transform the data to better fit.

## Methods

Over the course of our analysis, we experimented with several different preprocessing, feature extraction, and learning models in order to optimize the ROC curve produced by our training estimates. 

### Feature Extraction:

The majority of our analysis was spent optimizing the features we were able to extract from each picture. The initial model we started with simply extracted the average and standard deviation of the r,g, and b channels that each image could be broken down into. By itself, this produced an estimated AUC of .725. Initial efforts focused on varying channel combinations in order to gain more information about which features appeared to have the greatest impact on prediction. Unexpectedly, the blue channel appeared to have the least effect on performance despite most commercial solar panels primarily being colored blue or grey. 

As a result, we began to explore the HSV color space as a way to better differentiate hue differences from those produced by saturation or lighting effects. Visually, these effects of changing color spaces can be observed above in Figure 1. This had a marked impact on being able to differentiating greyer regions from the blueish-grey solar panel regions. After all, in rgb-space, the “greyness” of a color tends to be spread across all three channels. In hsv-space, one entire value is devoted to how grey a region appears to be. After creating new features with the average and variance of hue and saturation values across the various images, our AUC value improved to around 0.8. Incorporating value data had a minor negative impact. 

### Prescreening: 

After we had determined a baseline metric to compare to, we attempted to include additional prescreening techniques to further improve our estimates. One of our attempts at prescreening included detecting MSER features in order to more deeply analyze MSER objects. We took this approach because it had been used to great effect in Malof’s paper. Unfortunately, over the course of our analysis, we found that extracting MSER features had little to no impact on the accuracy of our predictions. Furthermore, visualization of the MSER data created notable discrepancies between perceived correctness and reported values. This persisted despite several attempts at tuning the input parameters to better target perceived solar panel properties. 

Figures 6 and 7, below, show sample outputs of our implementation of MSER. Notably, in HSV-space, fewer MSER regions overlap.

![image](https://raw.githubusercontent.com/y-ke/machine-learning-course/gh-pages/rgb_mser.jpg)
<center>Figure 6. MSER in RGB Space<center>

![image](https://raw.githubusercontent.com/y-ke/machine-learning-course/gh-pages/hsv_mser.jpg)
<center>Figure 7. MSER in HSV Space<center>

### Masking:

Due to our lack of success with MSER, we decided to attempt to implement masking a simplistic masking technique to determine regions of interest. After studying the data in HSV space, we noticed that on average, solar panels tended to have hue values between 0.5 and 0.6 on the 0-1 scale for representation of HSV color space. Additionally, the s values tended to be lower than 0.3. From our observations, pixels that satisfied both of those constraints were often synonymous with the presence of a solar panel. These pixels were then counted and used as a binary feature. Essentially, if there were seen to be a number of satisfactory pixels above a threshold (chosen here to be 550), the feature used in analysis was set to 1000. If the count fell below the threshold, the feature was set to 0. However, due to the simplistic construction, some non-solar panel pixels also fell into these categories, so it did not improve performance drastically. In addition, the whole masking process introduced a lot of potential for bias in our data. Most of the values used for thresholding h,v, and count data were determined by analyzing various training images and attempting to choose values that were present in the majority of positive examples and not present in the majority of negative examples. 

Another thresholding technique we attempted set slightly broader thresholds for h and v channel data, then either compressed the data to be within the thresholded range or shifted the data to be as far out of the given threshold range as possible. There was no noticeable difference in using any of these methods from the more defensible procedure of not thresholding. 

### Model Selection:

The features used in our model tended to be obtained from the entirety of a given image. As a result, we found K-Nearest Neighbors (KNN) models to be the most effective at translating our feature extractions into accurate predictions. Theoretically, this is because KNN in general is very effective at relating two objects with similar parameters. SVM and Logistic models showcased worse performance.  



## Results

### Cross Validation

In order to prevent overfitting of the model, we need comprehensive performance assessment of our model.  In this study, we chose cross validation as our validation approach. We set fold number at 20, which means that we would split training data set for 20 times and randomly select $\frac{1}{20}$ of data as test data each time. We use the rest $\frac{19}{20}$ of data to train our model and use ROC curve to assess the performance of our model on test data. The ROC curve would show the overall performance of the model in 20 times’ test.

### Performance assessment

Here we would show the performance of our model. To compare our results to other baselines more directly. We created two arrays, one with all negative outcomes(full of 0) and one which comes from *np.random.choice* function(randomly put in 1 or 0). The reason why we chose all negative array is that most of the images do not contain PV solar panels.
![image](https://raw.githubusercontent.com/y-ke/machine-learning-course/gh-pages/roc_real.png)
<center>Figure 8. ROC of our model<center>


![image](https://raw.githubusercontent.com/ys203/machine-learning-course/master/All%20negative%20ROC.png)
<center>Figure 9. ROC for random choice<center>

![image](https://raw.githubusercontent.com/ys203/machine-learning-course/master/random%20ROC.png)
<center>Figure 10. ROC for all negative choice<center>


 As shown in the curves above, our model shows a high AUC score (over 0.8) compared to baselines. Now, let’s look at several examples of accurate recognition as well as false recognition. (True positive: $p$>0.9; True negative: $p$=0; False positive: $p$>0.7; False negative: $p$<0.3) 


![image](https://raw.githubusercontent.com/ys203/machine-learning-course/master/Truepositive.png)
![image](https://raw.githubusercontent.com/ys203/machine-learning-course/master/Truenegative.png)
<center>Figure 11. Examples of accurate recognition (True Positive & True negative)<center>

![image](https://raw.githubusercontent.com/ys203/machine-learning-course/master/Flasepositive.png)
![image](https://raw.githubusercontent.com/ys203/machine-learning-course/master/Flaseneagtive.png)
<center>Figure 12. Examples of false recognition (False Positive & False negative)<center>

From examples above, we could see that our model works well on images with a large area of PV solar panels. However, the existence of road and shadow would influence recognition process. The reason might be that RGB or HSV model cannot distinguish panels from the roads as they look alike even through our eyes. In addition, the brighter roof color would prevent us from detecting panels. That may be due to that brightness of the image would affect h values of pixels.




## Conclusions

The primary challenge we have confronted with is how to extract useful features from the raw data, which have large dimension causing it difficult to handle them at the same time. Therefore, we observed the color pattern of some solar panels and made assumption about what could distinguish solar panels from their surroundings. The first idea came to mind was to construct possible RGB range of solar panels. Next, use some mask function in preprocessing step to filter out the colors outside the possible solar panel range, making the solar panels to “stand out” more in preprocessed images. 

Subsequently, we noticed that if features extracted from HSV channels performed more satisfactory features extracted from RGB channels. After we switched to using features extracted as mean and standard deviation from HSV channels, we experimented combinations of these features. Later on we included another feature counting how many pixels in one image falls into the possible solar panel color range in HSV channel. However, these method lacks a more accurate differentiate solar panels from blocks from background that has alike color, such as shadow, roads, and sometimes trees. Hence, we thought we could adopt the technique opted in the Automatic Solar Photovoltaic Panel Detection in Satellite Imagery paper, Maximum Stable extremal regions (MSER). Unfortunately tuning the parameters could not enable us to generate the result we wanted: the MSER method still missed some solar panels and falsely recognized blocks with similar color as solar panel. 

One takeaway from this competition is that to look at the data from a different angle might produce better result. The data was originally given in RGB channel, but our experience showed that HSV channel data made more significance in classification. Therefore in the future work, similar way of thinking could be applied. 

Another takeaway is that although different combinations could generate slightly different ROC curve result, a more precise way of extracting features is needed if we want a bottleneck breakthrough. Procedures such as Principal Component Analysis could help reducing the feature dimensions, algorithms such as MSER could help finding possible solar panel spots. 

One approach that worked well was to explore and compare the performance of different classification algorithms. By comparison, we kept using K-Nearest Neighbor classification algorithm because actually it worked better than Support Vector Machine and logistic regression.

In this competition, we have learned techniques with analyzing pictures and these techniques could help us open the door to other problems that involves with image processing. Next, we might be interested in the research of Convolutional Neural Network and Capsule Network, as they are deep neural network which has been effective and widely used in classifying images. 


## Roles

Hayden Bader: Extract features from RGB and HSV data; Assist in adjusting model parameters; Investigated thresholding-based prescreening; Write background and parts of methods for the report.

Yujing Ke: Investigated MSER regions detection; Analyzed potential color pattern of solar panels; Wrote conclusion part of the report.

Yangdi Shen: Extract features from RGB and HSV data and adjust parameters of the model for better performance; Visualize original data and results;  Write data and result part of the report. 


## References

Camilo, Joseph, Rui Wang, Leslie M. Collins, Kyle Bradbury, and Jordan M. Malof. “Application of a Semantic Segmentation Convolutional Neural Network for Accurate Automatic Detection and Mapping of Solar Photovoltaic Arrays in Aerial Imagery.” ArXiv:1801.04018 [Cs], January 11, 2018. http://arxiv.org/abs/1801.04018.

Castellanos, Sergio, Deborah A Sunter, and Daniel M Kammen. “Rooftop Solar Photovoltaic Potential in Cities: How Scalable Are Assessment Approaches?” Environmental Research Letters 12, no. 12 (December 1, 2017): 125005. https://doi.org/10.1088/1748-9326/aa7857.

“Detect MSER Features and Return MSERRegions Object - MATLAB DetectMSERFeatures.” Accessed March 7, 2018. https://www.mathworks.com/help/vision/ref/detectmserfeatures.html.

Malof, Jordan M., Rui Hou, Leslie M. Collins, Kyle Bradbury, and Richard Newell. “Automatic Solar Photovoltaic Panel Detection in Satellite Imagery,” 1428–31. IEEE, 2015. https://doi.org/10.1109/ICRERA.2015.7418643.

Google Project Sunroof. "Project Sunroof Data Explorer: a Description of Methodology and Inputs" Accessed March 7, 2018. https://www.google.com/get/sunroof/data-explorer/data-explorer-methodology.pdf.



